In [13]:
import sys
sys.path.append("..")

from dotenv import load_dotenv
import os
from sqlalchemy import create_engine,text
from dotenv import load_dotenv
import os
from utility.utils import get_response
from utility.utils import num_tokens_from_string
load_dotenv()
engine = create_engine(os.getenv("DATABASE_URL"),pool_pre_ping=True)
table_name = "sample_table2"
path = "data/big_sample.xlsx"
# upload_excel_to_psql(path = path,table_name=table_name,engine=engine)
# from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))



In [12]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_response_from_database",
            "description": "Fetches results from the postgre database assistant.",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {
                        "type": "string",
                        "description": "The result you want from the database in natural language",
                    }
                },
                "required": ["prompt"],
            },
        }
    }
]

def chat_completion_request(messages, tools=tools, tool_choice=None, model="gpt-3.5-turbo"):
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice=tool_choice,
    )
    return response


In [31]:
messages = []

system_prompt = """
You are a helpful assistant and have access to another database assistant.
Your job is to read the users questions and call the database assistant if needed.
Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

Here are the database fields present. The database assistant can only answer queries that revolve around these column names. If a user asks question outside of these fields do not call the database assistant.
('Year', 'bigint')
('COGS', 'double precision')
('Profit', 'double precision')
('Date', 'timestamp without time zone')
('Month Number', 'bigint')
('index', 'bigint')
('Units Sold', 'double precision')
('Manufacturing Price', 'bigint')
('Sale Price', 'bigint')
('Gross Sales', 'double precision')
('Discounts', 'double precision')
('Sales', 'double precision')
('Segment', 'text')
('Country', 'text')
('Product', 'text')
('Discount Band', 'text')
('Month Name', 'text')
"""
import json
import time
messages.append({"role": "system", "content": system_prompt})
print("Hi, how can I help you?")
while(True):
    question = input()
    print(question)
    if(question == "exit"):
        break
    messages.append({"role": "user", "content": question})
    chat_response = chat_completion_request(
        messages, tools=tools
    )
    assistant_message = chat_response.choices[0].message
    messages.append(assistant_message)
    tool_calls = assistant_message.tool_calls
    if(tool_calls):
        prompt = json.loads(assistant_message.tool_calls[0].function.arguments)["prompt"]
        function_response = get_response(question = prompt,table_name=table_name,engine=engine)
        token = num_tokens_from_string(str(function_response))
        if(token > 8000):
            function_response = "The database response is larger than the token limit. Please try Giving a shorter query."
        messages.append(
                {
                    "tool_call_id": tool_calls[0].id,
                    "role": "tool",
                    "name": tool_calls[0].function.name,
                    "content": function_response,
                }
        )
        messages.append({
            "role" : "assistant",
            "content" : function_response
        })
        print(function_response)
        continue
    print(assistant_message.content)
    time.sleep(0.3)

Hi, how can I help you?
Hi
Hello! How can I assist you today?
How is the weather today?
I'm here to help with database-related queries. If you have any questions or need data from the database fields I mentioned earlier, feel free to ask!
Can you list all the unique segments from the database?
The unique segments in the database are:
1. Enterprise
2. Small Business
3. Channel Partners
4. Midmarket
5. Government
How many segments are there?
There are a total of 5 segments in the database.
exit


In [32]:
messages

[{'role': 'system',
  'content': "\nYou are a helpful assistant and have access to another database assistant.\nYour job is to read the users questions and call the database assistant if needed.\nDon't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\n\nHere are the database fields present. The database assistant can only answer queries that revolve around these column names. If a user asks question outside of these fields do not call the database assistant.\n('Year', 'bigint')\n('COGS', 'double precision')\n('Profit', 'double precision')\n('Date', 'timestamp without time zone')\n('Month Number', 'bigint')\n('index', 'bigint')\n('Units Sold', 'double precision')\n('Manufacturing Price', 'bigint')\n('Sale Price', 'bigint')\n('Gross Sales', 'double precision')\n('Discounts', 'double precision')\n('Sales', 'double precision')\n('Segment', 'text')\n('Country', 'text')\n('Product', 'text')\n('Discount Band', 'text')\n('Month Na

In [14]:
a = chat_completion_request(messages=[{"role" : "user" , "content" : "Hi"}])

In [16]:
type(a)

openai.types.chat.chat_completion.ChatCompletion